In [1]:
!pip install pytube
!pip install pydub
!pip install torch torchvision torchaudio
!pip install ffmpeg-python
!pip install -U openai-whisper
!pip install setuptools-rust
!pip install ffmpy3
!pip install mock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798399 sha256=f24d691dd851baef0ee3998c0e99437215fac757fb952822a985695a569553e5
  Stored in directory: /root/.cache/pip/wheels/5d/37/b1/9aea93201fe91e3561719120da92cc23e77b7ef6f3d0d9491a
Successfully built openai-whisper


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import whisper
import numpy as np
from pytube import YouTube
from pydub import AudioSegment
from pydub.utils import get_array_type
import numpy as np
import io, datetime, whisper, os

In [5]:
import os, sqlite3

# base_path= os.path.abspath(os.path.dirname(__file__))#savelink的絕對路徑
createTableText= {
    "youtube":'''CREATE TABLE IF NOT EXISTS youtube(
        ID INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        description TEXT,
        link TEXT,
        company TEXT,
        videoContent TEXT
    );'''
}


def getData(tableName, commamd):
    conn = sqlite3.connect("/content/drive/MyDrive/data.db")
    cursor= conn.cursor()
    cursor.execute(createTableText[tableName])
    result= cursor.execute(commamd)
    # conn.close()
    return result.fetchall()


def insertData(tableName, data:dict):
    conn = sqlite3.connect("/content/drive/MyDrive/data.db")

    cursor= conn.cursor()
    cursor.execute(createTableText[tableName])

    values= list(data.values())
    for i,value in enumerate(values):
        if type(value)==str:
            values[i]= "'"+ value.replace("'","''")+ "'"
        elif type(value)==int:
            values[i]= str(value)
    cursor.execute(f"INSERT INTO {tableName} ({','.join(data.keys())}) VALUES ({','.join(values)});")
    new_data_id= cursor.lastrowid
    conn.commit()
    # conn.close()
    return new_data_id

def updatetData(tableName, data:dict,id):
    conn = sqlite3.connect("/content/drive/MyDrive/data.db")

    cursor= conn.cursor()
    cursor.execute(createTableText[tableName])

    values= list(data.values())
    for i,value in enumerate(values):
        if type(value)==str:
            values[i]= "'"+ value.replace("'","''")+ "'"
        elif type(value)==int:
            values[i]= str(value)
    cursor.execute(f"UPDATE {tableName} SET {','.join(data.keys())} = ({','.join(values)}) WHERE (id={id});")
    new_data_id= cursor.lastrowid
    conn.commit()
    # conn.close()
    return new_data_id

In [6]:
model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [00:56<00:00, 54.2MiB/s]


In [7]:
# pytube無法解析串流解決補釘
import mock
from pytube.cipher import get_throttling_function_code
def patched_throttling_plan(js: str):
    """Patch throttling plan, from https://github.com/pytube/pytube/issues/1498"""
    raw_code = get_throttling_function_code(js)

    transform_start = r"try{"
    plan_regex = re.compile(transform_start)
    match = plan_regex.search(raw_code)

    #transform_plan_raw = find_object_from_startpoint(raw_code, match.span()[1] - 1)
    transform_plan_raw = js

    # Steps are either c[x](c[y]) or c[x](c[y],c[z])
    step_start = r"c\[(\d+)\]\(c\[(\d+)\](,c(\[(\d+)\]))?\)"
    step_regex = re.compile(step_start)
    matches = step_regex.findall(transform_plan_raw)
    transform_steps = []
    for match in matches:
        if match[4] != '':
            transform_steps.append((match[0],match[1],match[4]))
        else:
            transform_steps.append((match[0],match[1]))
    return transform_steps

In [12]:
import re
def page(url):
    # 計算時間用1
    startTime= datetime.datetime.now()

    # pytube無法解析串流解決補釘(套用前面的補釘function)
    with mock.patch("pytube.cipher.get_throttling_plan", patched_throttling_plan):
        youtube= YouTube(url)
        audio= youtube.streams.filter(only_audio=True).first()

    # 下載成檔案(測試用)
    # out_file= audio.download()
    # fileName= os.path.splitext(out_file)
    # os.rename(out_file, "audio.mp3")

    # 將下載的串流轉為bytes物件
    bufferObj= io.BytesIO()
    audio.stream_to_buffer(bufferObj)
    # 將讀取記憶體的指標設回到可供讀取的預設位置(因為pytube的stream_to_buffer沒有寫上這個應該要有的功能，在此補上)
    bufferObj.seek(0)


    # 計算時間用2
    downloadTime= datetime.datetime.now()


    # 新增pydub要用的ffmpeg的路徑
    # os.environ['PATH'] += ';'+os.path.abspath(os.path.dirname(__file__)+'/data/ffmpeg/')

    audio = AudioSegment.from_file(bufferObj)

    # 從下載下來的檔案讀取(測試用)
    # audio = AudioSegment.from_file("audio.wav")

    # 16-bit(2bytes) pytube下載下來的貌似原本就是16bit的格式，可以省略
    # audio = audio.set_sample_width(2)
    # resampling 重設取樣率，whisper預設是以16000進行辨識
    audio = audio.set_frame_rate(16000)
    # 設定聲音為單聲道，避免雙聲道時，模型將雙聲道的編碼誤認為是一個雙倍長度的單聲道編碼
    audio= audio.set_channels(1)

    dtype = get_array_type(audio.sample_width * 8)
    # 轉換成np_array
    audio = np.frombuffer(audio.raw_data, dtype=dtype).astype(np.float32) / 32768.0

    # 將現在的音檔匯出成mp3(測試用)
    # audio.export("audio2.mp3", format="mp3")

    # model = whisper.load_model('tiny', download_root='data/whisper_model', device="cuda")
    result = model.transcribe(audio, language='zh', initial_prompt='請給我繁體中文的語音辨識')

    print("下載轉檔: ", (downloadTime-startTime).seconds, "秒\t文字轉換: ", (datetime.datetime.now()-downloadTime).seconds, "秒")
    return result["text"]

data = getData("youtube","SELECT link,id,videoContent FROM youtube")
for row in data:
  if not row[2]:#if row[2]沒東西
    updatetData("youtube", {"videoContent":page(row[0])}, row[1])

下載轉檔:  2 秒	文字轉換:  3 秒
下載轉檔:  3 秒	文字轉換:  22 秒
下載轉檔:  3 秒	文字轉換:  2 秒
下載轉檔:  3 秒	文字轉換:  118 秒
下載轉檔:  4 秒	文字轉換:  210 秒
下載轉檔:  3 秒	文字轉換:  310 秒
下載轉檔:  8 秒	文字轉換:  223 秒
下載轉檔:  3 秒	文字轉換:  129 秒
